# Exploratory Data Analysis - SPDX DocFest 2021-09-16

Author: Derek Kruszewski

In [399]:
import pandas as pd
import altair as alt
import json
import sys
import re

## Metadata Analysis from M. White

### 1. SBOM Type distribution?

In [144]:
def file_target(my_string):
    '''Determine SBOM target'''
    
    targets = {
        'time':'time',
        'App-BOM-ination':'app',
        'serve':'serve',
        'hello-word':'hello'
    }
    
    for t_name, t_search in targets.items():
        if re.match(f'.*{t_search.lower()}.*', my_string.lower()):
            return t_name
    return 'other'

In [145]:
# Load input
with open("Analysis/File analysis/sbom-data.json") as sbom_data_file:
    sbom_data_file_json = json.load(sbom_data_file)
df_sbom = pd.DataFrame(sbom_data_file_json)
# Determine SBOM target and Producer
df_sbom['target'] = df_sbom.file.apply(lambda x: file_target(x))
df_sbom['producer'] = df_sbom.file.apply(lambda x: x.split('/')[1])

In [146]:
df_sbom

,file,format,gzipped,standard,version,zipfile,target,producer
0,inputs-docfest/Source Auditor/time-v1.9/Source...,json,False,spdx,SPDX-2.2,False,time,Source Auditor
1,inputs-docfest/Source Auditor/time-v1.9/Source...,yaml,False,spdx,SPDX-2.2,False,time,Source Auditor
2,inputs-docfest/Source Auditor/time-v1.9/Source...,tv,False,spdx,2.2,False,time,Source Auditor
3,inputs-docfest/Source Auditor/time-v1.9/Source...,xml/rdf,False,spdx,SPDX-2.2,False,time,Source Auditor
4,inputs-docfest/Source Auditor/App-BOM-ination/...,json,False,spdx,SPDX-2.2,False,App-BOM-ination,Source Auditor
...,...,...,...,...,...,...,...,...
56,inputs-docfest/Zephyr West SPDX/zephyr-hello-w...,tv,False,spdx,2.2,False,hello-word,Zephyr West SPDX
57,inputs-docfest/Zephyr West SPDX/zephyr-hello-w...,tv,False,spdx,2.2,False,hello-word,Zephyr West SPDX
58,inputs-docfest/Zephyr West SPDX/zephyr-hello-w...,tv,False,spdx,2.2,False,App-BOM-ination,Zephyr West SPDX
59,inputs-docfest/Zephyr West SPDX/zephyr-hello-w...,tv,False,spdx,2.2,False,hello-word,Zephyr West SPDX


In [140]:
df_sbom.version.value_counts()

2.2         35
SPDX-2.2    26
Name: version, dtype: int64

In [141]:
print('Number of Submissions:', df_sbom.shape[0])

Number of Submissions: 61


In [142]:
df_sbom_summary = df_sbom.groupby(['target']).file.count().reset_index().rename(columns={'file':'file_count'})
df_sbom_summary['percent_total'] = round(df_sbom_summary.file_count/num_submissions,3)*100

In [143]:
df_sbom_summary

,target,file_count,percent_total
0,App-BOM-ination,17,27.9
1,hello-word,8,13.1
2,other,10,16.4
3,serve,4,6.6
4,time,22,36.1


### 2. Package Name Distribution?

In [401]:
# Load Input
with open("Analysis/File analysis/sbom-report.json") as sbom_data_report:
    sbom_data_report_json = json.load(sbom_data_report)
df_sbom_data = pd.DataFrame(sbom_data_report_json)
# Merge Sbom details
df_sbom_data = df_sbom_data.merge(df_sbom, how='left', left_on='bom', right_on='file')
# Obtain G.ONeall Groupings
sbom_types = pd.read_csv('sbom_groups.csv')
sbom_types['bom'] = sbom_types.sbom.apply(lambda x: 'inputs-docfest/'+x)
# Merge G.ONeall Groupings
df_sbom_data = df_sbom_data.merge(sbom_types, how='left', on='bom')

In [402]:
# Filter to unique SBOM producers (ignore multiple SBOMs submitted 1 producers)
df_sbom_data_packages = df_sbom_data.explode('package_names')
df_sbom_data_packages = df_sbom_data_packages.drop_duplicates(subset=['package_names','producer'])
df_sbom_data_packages = df_sbom_data_packages.dropna(subset=['package_names'])

In [406]:
df_sbom_data_packages[df_sbom_data_packages.bom == 'inputs-docfest/REA_SAG-PM/SAG-PM-time1_9_Source_Tar.spdx']

,bom,files,files_with_hashes,got_final_product,package_names,packages,packages_with_hashes,packages_with_ids,packages_with_names,packages_with_suppliers,...,file,format,gzipped,standard,version,zipfile,target,producer,sbom,type_y
12,inputs-docfest/REA_SAG-PM/SAG-PM-time1_9_Sourc...,1,1.0,True,time-1.9,1,1,1,1,1,...,inputs-docfest/REA_SAG-PM/SAG-PM-time1_9_Sourc...,tv,False,spdx,2.2,False,time,REA_SAG-PM,REA_SAG-PM/SAG-PM-time1_9_Source_Tar.spdx,source


#### Time

In [403]:
# Filter data
chart_data = df_sbom_data_packages[(df_sbom_data_packages.target == 'time') & (df_sbom_data_packages.type_y == 'binary')]
chart_data_grouped = chart_data.groupby(['package_names']).count().reset_index().sort_values('package_names').sort_values('bom', ascending=False)
# Plot data
pm1 = alt.Chart(chart_data_grouped, title='Package Frequency').mark_bar(color='orange').encode(
    x = alt.X('bom', title='Count', axis=alt.Axis(tickMinStep=1), scale=alt.Scale(domain=(0,len(chart_data.producer.unique())+1))),
    y = alt.Y('package_names', sort='-x', title=None)
)
rule = alt.Chart(chart_data.drop_duplicates(subset=['producer'])).mark_rule(color='red').encode(
    x=('count(producer)')
)
(pm1+rule).properties(height=100,width=400).configure_axis(
    labelFontSize=20,
    titleFontSize=15
).configure_title(fontSize=20)

alt.LayerChart(...)

In [190]:
# Filter data
chart_data = df_sbom_data_packages[(df_sbom_data_packages.target == 'time') & (df_sbom_data_packages.type_y == 'source')]
chart_data_grouped = chart_data.groupby(['package_names']).count().reset_index().sort_values('package_names').sort_values('bom', ascending=False)
# Plot data
pm1 = alt.Chart(chart_data_grouped, title='Package Frequency').mark_bar(color='orange').encode(
    x = alt.X('bom', title='Count', axis=alt.Axis(tickMinStep=1), scale=alt.Scale(domain=(0,len(chart_data.producer.unique())+1))),
    y = alt.Y('package_names', sort='-x', title=None)
)
rule = alt.Chart(chart_data.drop_duplicates(subset=['producer'])).mark_rule(color='red').encode(
    x=('count(producer)')
)
(pm1+rule).properties(height=300,width=400).configure_axis(
    labelFontSize=20,
    titleFontSize=15
).configure_title(fontSize=20)

alt.LayerChart(...)

In [469]:
# Filter data
chart_data = df_sbom_data_packages[(df_sbom_data_packages.target == 'time') & (df_sbom_data_packages.type_y == 'container')]
chart_data_grouped = chart_data.groupby(['package_names']).count().reset_index().sort_values('package_names').sort_values('bom', ascending=False)
# Plot data
pm1 = alt.Chart(chart_data_grouped, title='Package Frequency').mark_bar(color='orange').encode(
    x = alt.X('bom', title='Count', axis=alt.Axis(tickMinStep=1), scale=alt.Scale(domain=(0,len(chart_data.producer.unique())+1))),
    y = alt.Y('package_names', sort='-x', title=None, axis=alt.Axis(labels=False))
)
rule = alt.Chart(chart_data.drop_duplicates(subset=['producer'])).mark_rule(color='red').encode(
    x=('count(producer)')
)
(pm1+rule).properties(height=600,width=400).configure_axis(
    labelFontSize=20,
    titleFontSize=15
).configure_title(fontSize=20)

alt.LayerChart(...)

In [480]:
# Filter data
chart_data = df_sbom_data_packages[(df_sbom_data_packages.target == 'time') & (df_sbom_data_packages.type_y == 'container')]
chart_data_grouped = chart_data.groupby(['package_names']).count().reset_index().sort_values('package_names').sort_values('bom', ascending=False).head(40)
# Plot data
pm1 = alt.Chart(chart_data_grouped, title='Package Frequency').mark_bar(color='orange').encode(
    x = alt.X('bom', title='Count', axis=alt.Axis(tickMinStep=1), scale=alt.Scale(domain=(0,len(chart_data.producer.unique())+1))),
    y = alt.Y('package_names', sort='-x', title=None)#, axis=alt.Axis(labels=False))
)
rule = alt.Chart(chart_data.drop_duplicates(subset=['producer'])).mark_rule(color='red').encode(
    x=('count(producer)')
)
(pm1+rule).properties(height=500,width=400).configure_axis(
    labelFontSize=20,
    titleFontSize=15
).configure_title(fontSize=20)

alt.LayerChart(...)

#### App-BOM-ination

In [488]:
# Filter data
chart_data = df_sbom_data_packages[(df_sbom_data_packages.target == 'App-BOM-ination')]
chart_data_grouped = chart_data.groupby(['package_names']).count().reset_index().sort_values('package_names').sort_values('bom', ascending=False)
# Plot data
pm1 = alt.Chart(chart_data_grouped, title='Package Frequency').mark_bar(color='orange').encode(
    x = alt.X('bom', title='Count', axis=alt.Axis(tickMinStep=1), scale=alt.Scale(domain=(0,len(chart_data.producer.unique())+1))),
    y = alt.Y('package_names', sort='-x', title=None)
)
rule = alt.Chart(chart_data.drop_duplicates(subset=['producer'])).mark_rule(color='red').encode(
    x=('count(producer)')
)
(pm1+rule).properties(height=600,width=400).configure_axis(
    labelFontSize=20,
    titleFontSize=15
).configure_title(fontSize=20)

alt.LayerChart(...)

#### Serve

In [495]:
# Filter data
chart_data = df_sbom_data_packages[(df_sbom_data_packages.target == 'serve')]
chart_data_grouped = chart_data.groupby(['package_names']).count().reset_index().sort_values('package_names').sort_values('bom', ascending=False)
# Plot data
pm1 = alt.Chart(chart_data_grouped, title='Package Frequency').mark_bar(color='orange').encode(
    x = alt.X('bom', title='Count', axis=alt.Axis(tickMinStep=1), scale=alt.Scale(domain=(0,len(chart_data.producer.unique())+1))),
    y = alt.Y('package_names', sort='-x', title=None)
)
rule = alt.Chart(chart_data.drop_duplicates(subset=['producer'])).mark_rule(color='red').encode(
    x=('count(producer)')
)
(pm1+rule).properties(height=400,width=400).configure_axis(
    labelFontSize=20,
    titleFontSize=15
).configure_title(fontSize=20)

alt.LayerChart(...)

### 3. File Checksum Distribution?

In [451]:
# Load data
with open("Analysis/File analysis/sbom-hashes.json") as sbom_data_hash:
    sbom_data_hash_json = json.load(sbom_data_hash)
df_sbom_hashes = pd.DataFrame({'bom':sbom_data_hash_json.keys(), 'hash':sbom_data_hash_json.values()})

# Merge sbom information
df_sbom_hashes = df_sbom_hashes.merge(df_sbom, how='left', left_on='bom', right_on='file')
df_sbom_hashes = df_sbom_hashes.merge(sbom_types, how='left', on='bom')

df_sbom_hashes = df_sbom_hashes.explode('hash')#.dropna()
df_sbom_hashes.hash=df_sbom_hashes.hash.str.lower()
df_sbom_hashes_unique = df_sbom_hashes.drop_duplicates(subset=['hash','producer'])

#### Time

In [458]:
# Filter data
chart_data = df_sbom_hashes_unique[(df_sbom_hashes_unique.target == 'time') & (df_sbom_hashes_unique.type == 'binary')]
chart_data_grouped = chart_data.groupby(['hash']).count().reset_index().sort_values('hash').sort_values('bom', ascending=False)
# Plot data
pm1 = alt.Chart(chart_data_grouped, title='File Frequency').mark_bar(color='orange').encode(
    x = alt.X('bom', title='Count', axis=alt.Axis(tickMinStep=1), scale=alt.Scale(domain=(0,len(chart_data.producer.unique())+1))),
    y = alt.Y('hash', sort='-x', title=None)
)
rule = alt.Chart(chart_data.drop_duplicates(subset=['producer'])).mark_rule(color='red').encode(
    x=('count(producer)')
)
(pm1+rule).properties(height=25,width=400).configure_axis(
    labelFontSize=20,
    titleFontSize=15
).configure_title(fontSize=20)

alt.LayerChart(...)

In [471]:
# Filter data
chart_data = df_sbom_hashes_unique[(df_sbom_hashes_unique.target == 'time') & (df_sbom_hashes_unique.type == 'source')]
chart_data_grouped = chart_data.groupby(['hash']).count().reset_index().sort_values('hash').sort_values('bom', ascending=False)
# Plot data
pm1 = alt.Chart(chart_data_grouped, title='File Frequency').mark_bar(color='orange').encode(
    x = alt.X('bom', title='Count', axis=alt.Axis(tickMinStep=1), scale=alt.Scale(domain=(0,len(chart_data.producer.unique())+1))),
    y = alt.Y('hash', sort='-x', title=None, axis=alt.Axis(labels=False))
)
rule = alt.Chart(chart_data.drop_duplicates(subset=['producer'])).mark_rule(color='red').encode(
    x=('count(producer)')
)
(pm1+rule).properties(height=400,width=400).configure_axis(
    labelFontSize=20,
    titleFontSize=15
).configure_title(fontSize=20)

alt.LayerChart(...)

In [468]:
# Filter data
chart_data = df_sbom_hashes_unique[(df_sbom_hashes_unique.target == 'time') & (df_sbom_hashes_unique.type == 'container')]
chart_data_grouped = chart_data.groupby(['hash']).count().reset_index().sort_values('hash').sort_values('bom', ascending=False)
# Plot data
pm1 = alt.Chart(chart_data_grouped, title='File Frequency').mark_bar(color='orange').encode(
    x = alt.X('bom', title='Count', axis=alt.Axis(tickMinStep=1), scale=alt.Scale(domain=(0,len(chart_data.producer.unique())+1))),
    y = alt.Y('hash', sort='-x', title=None, axis=alt.Axis(labels=False))
)
rule = alt.Chart(chart_data.drop_duplicates(subset=['producer'])).mark_rule(color='red').encode(
    x=('count(producer)')
)
(pm1+rule).properties(height=400,width=400).configure_axis(
    labelFontSize=20,
    titleFontSize=15
).configure_title(fontSize=20)

C:\ProgramData\Anaconda3\lib\site-packages\altair\utils\core.py:185: UserWarning: I don't know how to infer vegalite type from 'empty'.  Defaulting to nominal.
  warnings.warn(


alt.LayerChart(...)

#### App-BOM-ination

In [455]:
# Filter data
chart_data = df_sbom_hashes_unique[(df_sbom_hashes_unique.target == 'App-BOM-ination')]
chart_data_grouped = chart_data.groupby(['hash']).count().reset_index().sort_values('hash').sort_values('bom', ascending=False)
# Plot data
pm1 = alt.Chart(chart_data_grouped, title='File Frequency').mark_bar(color='orange').encode(
    x = alt.X('bom', title='Count', axis=alt.Axis(tickMinStep=1), scale=alt.Scale(domain=(0,len(chart_data.producer.unique())+1))),
    y = alt.Y('hash', sort='-x', title=None, axis=alt.Axis(labels=False))
)
rule = alt.Chart(chart_data.drop_duplicates(subset=['producer'])).mark_rule(color='red').encode(
    x=('count(producer)')
)
(pm1+rule).properties(height=400,width=400).configure_axis(
    labelFontSize=20,
    titleFontSize=15
).configure_title(fontSize=20)

alt.LayerChart(...)

#### Serve

In [499]:
# Filter data
chart_data = df_sbom_hashes_unique[(df_sbom_hashes_unique.target == 'serve')]
chart_data_grouped = chart_data.groupby(['hash']).count().reset_index().sort_values('hash').sort_values('bom', ascending=False)
# Plot data
pm1 = alt.Chart(chart_data_grouped, title='File Frequency').mark_bar(color='orange').encode(
    x = alt.X('bom', title='Count', axis=alt.Axis(tickMinStep=1), scale=alt.Scale(domain=(0,len(chart_data.producer.unique())+1))),
    y = alt.Y('hash', sort='-x', title=None)
)
rule = alt.Chart(chart_data.drop_duplicates(subset=['producer'])).mark_rule(color='red').encode(
    x=('count(producer)')
)
(pm1+rule).properties(height=200,width=400).configure_axis(
    labelFontSize=20,
    titleFontSize=15
).configure_title(fontSize=20)

alt.LayerChart(...)

---

## Comparison Spreadsheets from G. ONeall

### 1. Package Name Distribution?

In [411]:
# Load data
packages_dict = {
    'time-b': pd.read_excel('Analysis/time-binary-compare.xlsx', sheet_name='Package'),
    'time-s': pd.read_excel('Analysis/time-source-compare.xlsx', sheet_name='Package'),
    'time-c': pd.read_excel('Analysis/time-container-compare.xlsx', sheet_name='Package'),
    'App-BOM-ination': pd.read_excel('Analysis/App-BOM-ination-compare.xlsx', sheet_name='Package'),
    'serve': pd.read_excel('Analysis/serve-compare.xlsx', sheet_name='Package')
}

In [412]:
df_packages = None
for k, v in packages_dict.items():
    v = v[v['Package Property'] == 'Package Name']
    v = pd.melt(v, id_vars=['Package Property'], value_vars=v.columns[~v.columns.isin(['Package Property', 'Equals'])].tolist())
    v = v[v.value != '[No Package]']
    v = v.drop(columns=['Package Property'])
    v['target'] = k
    if df_packages is None:
        df_packages = v
    else:
        df_packages = pd.concat([df_packages, v])

In [416]:
df_packages.loc[df_packages.target.isin(['time-b','time-s','time-c']),'target'] = 'time'

In [417]:
# Obtain Packages
df_packages['producer'] = df_packages.variable.apply(lambda x: x.split('/')[0])
# Remove producer duplicates 
df_packages = df_packages.drop_duplicates(subset=['value','producer', 'target'])
# Merge G. ONeall Groupings
df_packages = df_packages.merge(sbom_types, how='left', left_on='variable', right_on='sbom')

In [418]:
df_packages[df_packages.value == 'time-1.9']

,variable,value,target,producer,sbom,type,bom
4,metaeffekt/time-1.9/time-1.9.tar.gz.spdx.rdf.xml,time-1.9,time,metaeffekt,metaeffekt/time-1.9/time-1.9.tar.gz.spdx.rdf.xml,source,inputs-docfest/metaeffekt/time-1.9/time-1.9.ta...
5,Canvass Labs/time-1.9/CanvassLabs-time-1.9.spd...,time-1.9,time,Canvass Labs,Canvass Labs/time-1.9/CanvassLabs-time-1.9.spd...,source,inputs-docfest/Canvass Labs/time-1.9/CanvassLa...
6,FOSSology/time-1.9/SPDX2TV_time-1.9.tar.gz_162...,time-1.9,time,FOSSology,FOSSology/time-1.9/SPDX2TV_time-1.9.tar.gz_162...,source,inputs-docfest/FOSSology/time-1.9/SPDX2TV_time...
7,REA_SAG-PM/SAG-PM-time1_9_Source_Tar.spdx,time-1.9,time,REA_SAG-PM,REA_SAG-PM/SAG-PM-time1_9_Source_Tar.spdx,source,inputs-docfest/REA_SAG-PM/SAG-PM-time1_9_Sourc...
19,Source Auditor/time-v1.9/SourceAuditor-time-1....,time-1.9,time,Source Auditor,Source Auditor/time-v1.9/SourceAuditor-time-1....,source,inputs-docfest/Source Auditor/time-v1.9/Source...
20,Toshiba/time-1.9/SPDX2TV_time-1.9.tar.gz_16307...,time-1.9,time,Toshiba,Toshiba/time-1.9/SPDX2TV_time-1.9.tar.gz_16307...,source,inputs-docfest/Toshiba/time-1.9/SPDX2TV_time-1...
215,Tern/time/spdx_json/tern-time-scancode-spdx.json,time-1.9,time,Tern,Tern/time/spdx_json/tern-time-scancode-spdx.json,container,inputs-docfest/Tern/time/spdx_json/tern-time-s...


#### Time

In [420]:
chart_data = df_packages[(df_packages.target == 'time') & (df_packages.type == 'binary')]
chart_data_group = chart_data.groupby(['value']).count().reset_index().sort_values('value').sort_values('variable', ascending=False)
p1 = alt.Chart(chart_data_group, title='Package Frequency').mark_bar().encode(
    x = alt.X('variable', title='Count', axis=alt.Axis(tickMinStep=1), scale=alt.Scale(domain=(0,len(chart_data.producer.unique())+1))),
    y = alt.Y('value', sort='-x', title=None)
)
rule = alt.Chart(chart_data.drop_duplicates(subset=['producer'])).mark_rule(color='red').encode(
    x=('count(variable)')
)
(p1+rule).properties(height=50,width=400).configure_axis(
    labelFontSize=20,
    titleFontSize=15
).configure_title(fontSize=20)

alt.LayerChart(...)

In [421]:
chart_data = df_packages[(df_packages.target == 'time') & (df_packages.type == 'source')]
chart_data_group = chart_data.groupby(['value']).count().reset_index().sort_values('value').sort_values('variable', ascending=False)
p1 = alt.Chart(chart_data_group, title='Package Frequency').mark_bar().encode(
    x = alt.X('variable', title='Count', axis=alt.Axis(tickMinStep=1), scale=alt.Scale(domain=(0,len(chart_data.producer.unique())+1))),
    y = alt.Y('value', sort='-x', title=None)
)
rule = alt.Chart(chart_data.drop_duplicates(subset=['producer'])).mark_rule(color='red').encode(
    x=('count(variable)')
)
(p1+rule).properties(height=300,width=400).configure_axis(
    labelFontSize=20,
    titleFontSize=15
).configure_title(fontSize=20)

alt.LayerChart(...)

In [487]:
chart_data = df_packages[(df_packages.target == 'time') & (df_packages.type == 'container')]
chart_data_group = chart_data.groupby(['value']).count().reset_index().sort_values('value').sort_values('variable', ascending=False).head(40)
p1 = alt.Chart(chart_data_group, title='Package Frequency').mark_bar().encode(
    x = alt.X('variable', title='Count', axis=alt.Axis(tickMinStep=1), scale=alt.Scale(domain=(0,len(chart_data.producer.unique())+1))),
    y = alt.Y('value', sort='-x', title=None)#, axis=alt.Axis(labels=False))
)
rule = alt.Chart(chart_data.drop_duplicates(subset=['producer'])).mark_rule(color='red').encode(
    x=('count(variable)')
)
(p1+rule).properties(height=600,width=400).configure_axis(
    labelFontSize=20,
    titleFontSize=15
).configure_title(fontSize=20)

alt.LayerChart(...)

#### App-BOM-ination

In [423]:
chart_data = df_packages[(df_packages.target == 'App-BOM-ination')]
chart_data_group = chart_data.groupby(['value']).count().reset_index().sort_values('value').sort_values('variable', ascending=False)
p1 = alt.Chart(chart_data_group, title='Package Frequency').mark_bar().encode(
    x = alt.X('variable', title='Count', axis=alt.Axis(tickMinStep=1), scale=alt.Scale(domain=(0,len(chart_data.producer.unique())+1))),
    y = alt.Y('value', sort='-x', title=None)
)
rule = alt.Chart(chart_data.drop_duplicates(subset=['producer'])).mark_rule(color='red').encode(
    x=('count(variable)')
)
(p1+rule).properties(height=600,width=400).configure_axis(
    labelFontSize=20,
    titleFontSize=15
).configure_title(fontSize=20)

alt.LayerChart(...)

#### Serve

In [498]:
chart_data = df_packages[(df_packages.target == 'serve')]
chart_data_group = chart_data.groupby(['value']).count().reset_index().sort_values('value').sort_values('variable', ascending=False)
p1 = alt.Chart(chart_data_group, title='Package Frequency').mark_bar().encode(
    x = alt.X('variable', title='Count', axis=alt.Axis(tickMinStep=1), scale=alt.Scale(domain=(0,len(chart_data.producer.unique())+1))),
    y = alt.Y('value', sort='-x', title=None)
)
rule = alt.Chart(chart_data.drop_duplicates(subset=['producer'])).mark_rule(color='red').encode(
    x=('count(variable)')
)
(p1+rule).properties(height=200,width=400).configure_axis(
    labelFontSize=20,
    titleFontSize=15
).configure_title(fontSize=20)

alt.LayerChart(...)

### 2. File Checksum Distribution?

In [425]:
# Load data
file_checksum_dict = {
    'time': pd.read_excel('Analysis/time-compare-all.xlsx', sheet_name='File Checksum'),
    'App-BOM-ination': pd.read_excel('Analysis/App-BOM-ination-compare.xlsx', sheet_name='File Checksum'),
    'serve': pd.read_excel('Analysis/serve-compare.xlsx', sheet_name='File Checksum')
}

In [426]:
df_file_checksum = None
for k, v in file_checksum_dict.items():
    v = pd.melt(v, id_vars=['File Path'], value_vars=v.columns[~v.columns.isin(['File Path', 'Same/Diff'])].tolist())
    v = v[v.value != '[No File]']
    v = v.dropna()
    v['target'] = k
    if df_file_checksum is None:
        df_file_checksum = v
    else:
        df_file_checksum = pd.concat([df_file_checksum, v])

In [432]:
df_file_checksum.value = df_file_checksum.value.str.lower()

In [433]:
df_file_checksum.groupby(['variable', 'value']).count().reset_index().sort_values('File Path', ascending=False).head(20)

,variable,value,File Path,target
403,REA_SAG-PM/SAG-PM-time1_9_Source_Tar.spdx,2dc6b69d572ae23f6b8f9dae77feea7260db639b,171,171
5193,Tern/time/spdx_json/tern-time-scancode-spdx.json,8dae58cef087a99d2fae9403c50f0a9683a9624f,33,33
401,REA_SAG-PM/SAG-PM-SBOM-AppBom.spdx,c473018aaf2b8c5c45f7f8a035f0ff571a69eaba,25,25
7382,Tern/time/spdx_json/tern-time-scancode-spdx.json,d189fb1bf8491582d6e8372db76c20a98b53aeec,21,21
8430,Tern/time/spdx_json/tern-time-scancode-spdx.json,f0f97e93ac67c2f5bf8fd77bcd9c14eff4be173f,17,17
5842,Tern/time/spdx_json/tern-time-scancode-spdx.json,a19f080a0d23acf4ce0a56e30f63a5f213960e71,13,13
3101,Tern/time/spdx_json/tern-time-scancode-spdx.json,4df34d59dfeb44f70569075cf7829708e09c8aaa,11,11
8770,Tern/time/spdx_json/tern-time-scancode-spdx.json,fc01fbfa1d23bc4c9945d4f13548ff64837bdc0d,9,9
2262,Tern/time/spdx_json/tern-time-scancode-spdx.json,34a1e6b162012a3d6e50d59cd0ab1c096a8825dd,9,9
6350,Tern/time/spdx_json/tern-time-scancode-spdx.json,b0d6c1ee6ae5779cb634f2e65717725201cd53c0,9,9


In [434]:
# Get Producer
df_file_checksum['producer'] = df_file_checksum.variable.apply(lambda x: x.split('/')[0])
# Remove producer duplicates 
df_file_checksum = df_file_checksum.drop_duplicates(subset=['value','producer'])
# Merge G. ONeall Groupings
df_file_checksum = df_file_checksum.merge(sbom_types, how='left', left_on='variable', right_on='sbom')

#### Time

In [463]:
df_file_checksum[(df_file_checksum.target == 'time') & (df_file_checksum.type == 'binary')]

,File Path,variable,value,target,producer,sbom,type,bom
0,./time-1.9.tar.gz,metaeffekt/time-1.9/time-1.9.tar.gz_concluded-...,75068c26abbed3ad3980685bae21d7202d288317,time,metaeffekt,metaeffekt/time-1.9/time-1.9.tar.gz_concluded-...,binary,inputs-docfest/metaeffekt/time-1.9/time-1.9.ta...
493,./time.exe,REA_SAG-PM/SAG-PM-SBOM-Time_1_9.spdx,3c973008b1a7b6c628264da06ff54e150d913820,time,REA_SAG-PM,REA_SAG-PM/SAG-PM-SBOM-Time_1_9.spdx,binary,inputs-docfest/REA_SAG-PM/SAG-PM-SBOM-Time_1_9...


In [435]:
# Filter data
chart_data = df_file_checksum[(df_file_checksum.target == 'time') & (df_file_checksum.type == 'binary')]
chart_data_grouped = chart_data.groupby(['value']).count().reset_index().sort_values('value').sort_values('bom', ascending=False)
# Plot data
pm1 = alt.Chart(chart_data_grouped, title='File Frequency').mark_bar().encode(
    x = alt.X('variable', title='Count', axis=alt.Axis(tickMinStep=1), scale=alt.Scale(domain=(0,len(chart_data.producer.unique())+1))),
    y = alt.Y('value', sort='-x', title=None)
)
rule = alt.Chart(chart_data.drop_duplicates(subset=['producer'])).mark_rule(color='red').encode(
    x=('count(producer)')
)
(pm1+rule).properties(height=50,width=400).configure_axis(
    labelFontSize=20,
    titleFontSize=15
).configure_title(fontSize=20)

alt.LayerChart(...)

In [489]:
# Filter data
chart_data = df_file_checksum[(df_file_checksum.target == 'time') & (df_file_checksum.type == 'source')]
chart_data_grouped = chart_data.groupby(['value']).count().reset_index().sort_values('value').sort_values('bom', ascending=False)
# Plot data
pm1 = alt.Chart(chart_data_grouped, title='File Frequency').mark_bar().encode(
    x = alt.X('variable', title='Count', axis=alt.Axis(tickMinStep=1), scale=alt.Scale(domain=(0,len(chart_data.producer.unique())+1))),
    y = alt.Y('value', sort='-x', title=None, axis=alt.Axis(labels=False))
)
rule = alt.Chart(chart_data.drop_duplicates(subset=['producer'])).mark_rule(color='red').encode(
    x=('count(producer)')
)
(pm1+rule).properties(height=400,width=400).configure_axis(
    labelFontSize=20,
    titleFontSize=15
).configure_title(fontSize=20)

alt.LayerChart(...)

In [481]:
# Filter data
chart_data = df_file_checksum[(df_file_checksum.target == 'time') & (df_file_checksum.type == 'container')]
chart_data_grouped = chart_data.groupby(['value']).count().reset_index().sort_values('value').sort_values('bom', ascending=False).head(1000)
# Plot data
pm1 = alt.Chart(chart_data_grouped, title='File Frequency').mark_bar().encode(
    x = alt.X('variable', title='Count', axis=alt.Axis(tickMinStep=1), scale=alt.Scale(domain=(0,len(chart_data.producer.unique())+1))),
    y = alt.Y('value', sort='-x', title=None, axis=alt.Axis(labels=False))
)
rule = alt.Chart(chart_data.drop_duplicates(subset=['producer'])).mark_rule(color='red').encode(
    x=('count(producer)')
)
(pm1+rule).properties(height=400,width=400).configure_axis(
    labelFontSize=20,
    titleFontSize=15
).configure_title(fontSize=20)

alt.LayerChart(...)

#### App-BOM-ination

In [493]:
# Filter data
chart_data = df_file_checksum[(df_file_checksum.target == 'App-BOM-ination')]
chart_data_grouped = chart_data.groupby(['value']).count().reset_index().sort_values('value').sort_values('bom', ascending=False)
# Plot data
pm1 = alt.Chart(chart_data_grouped, title='File Frequency').mark_bar().encode(
    x = alt.X('variable', title='Count', axis=alt.Axis(tickMinStep=1), scale=alt.Scale(domain=(0,len(chart_data.producer.unique())+1))),
    y = alt.Y('value', sort='-x', title=None,  axis=alt.Axis(labels=False))
)
rule = alt.Chart(chart_data.drop_duplicates(subset=['producer'])).mark_rule(color='red').encode(
    x=('count(producer)')
)
(pm1+rule).properties(height=400,width=400).configure_axis(
    labelFontSize=20,
    titleFontSize=15
).configure_title(fontSize=20)

alt.LayerChart(...)

#### Serve

In [439]:
# Filter data
chart_data = df_file_checksum[(df_file_checksum.target == 'serve')]
chart_data_grouped = chart_data.groupby(['value']).count().reset_index().sort_values('value').sort_values('bom', ascending=False)
# Plot data
pm1 = alt.Chart(chart_data_grouped, title='File Frequency').mark_bar().encode(
    x = alt.X('variable', title='Count', axis=alt.Axis(tickMinStep=1), scale=alt.Scale(domain=(0,len(chart_data.producer.unique())+1))),
    y = alt.Y('value', sort='-x', title=None)
)
rule = alt.Chart(chart_data.drop_duplicates(subset=['producer'])).mark_rule(color='red').encode(
    x=('count(producer)')
)
(pm1+rule).properties(height=200,width=400).configure_axis(
    labelFontSize=20,
    titleFontSize=15
).configure_title(fontSize=20)

alt.LayerChart(...)